In [113]:
import pandas as pd
import numpy as np


df = pd.read_csv('data.csv')

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045


In [118]:
# data check + define features and target

# df.isna().sum()

x = df[['bedrooms','sqft_living','sqft_lot','floors','condition','grade','sqft_above']]
y = df['price']

# train test split

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y.values.reshape(-1,1),test_size=0.2)

# feature scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_train)
y_scaled = scaler.fit_transform(y_train)





In [122]:
# MODEL training 

from sklearn.linear_model import SGDRegressor

SGD = SGDRegressor()

SGD.fit(x_scaled,y_scaled)

#  model evaluation 

from sklearn.metrics import mean_squared_error

SGD.score(x_test,y_test)




print(mean_squared_error(y_test,SGD.predict(x_test),squared=False))
print(SGD.score(x_test,y_test))

# save model 

import pickle

pickle.dump(SGD,open('model.pkl','wb'))





640555.0990612045
-2.4720791708469187


/home/saber/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/saber/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
/home/saber/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(
/home/saber/.local/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SGDRegressor was fitted without feature names
  warnings.warn(


In [123]:
# flask API

from flask import Flask, request
import pickle
import numpy as np
app = Flask('__name__')
model=pickle.load(open('model.pkl','rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=["POST"])
def predict():
    feature=[int(x) for x in request.form.values()]
    #print(feature)
    feature_final=np.array(feature).reshape(-1,1)
    #print(feature_final)
    prediction=model.predict(feature_final)
    return render_template('index.html',prediction_text="The house's Price will be : {}".format(int(prediction)))

if(__name__=='__main__'):
    app.run(debug=True)



 * Serving Flask app '__name__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
/home/saber/.local/lib/python3.8/site-packages/traitlets/traitlets.py:2392: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/saber/.local/lib/python3.8/site-packages/traitlets/traitlets.py:2346: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'c8783b48-3b1a-4423-a135-bd5bf8a76753' instead of 'b"c8783b48-3b1a-4423-a135-bd5bf8a76753"'.
  warn(
Traceback (most recent call last):
  File "/home/saber/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/saber/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 975, in launch_instance
    app.initialize(argv)
  File "/home/saber/.local/lib/python3.8/site-packages/traitlets/config/application.py", line

SystemExit: 1

/home/saber/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
